In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [2]:
# Function to extract Product Title
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'})
        
        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

# Function to extract Product Price
def get_price(soup):

    try:
        # OG Price
        price = soup.find("span", attrs={"class":'a-price a-text-price a-size-base'}).find("span", attrs={"class":'a-offscreen'}).string.strip() 

    except AttributeError:

        try:
            # Sale Price
            price = soup.find("span", attrs={"class":'a-price aok-align-center'}).find("span", attrs={"class":'a-offscreen'}).string.strip() 

        except:
            price = ""

    return price

# Function to extract Product Rating
def get_rating(soup):

    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()
    
    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""	

    return rating

# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""	

    return review_count

# Function to extract Availability Status
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        available = "Not Available"	

    return available



In [3]:

if __name__ == '__main__':

    HEADERS = ({'User-Agent':'', 'Accept-Language': 'en-US, en;q=0.5'})
    # User-Agent 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36 Edg/121.0.0.0

    # The webpage URL
    URL = "https://www.amazon.com/s?k=Apple+iPhone"

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class':'a-link-normal s-no-outline'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[]}
    
    # Loop for extracting product details from each link 
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.com" + link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))

    
    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'] = amazon_df['title'].replace('', np.nan)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)


In [4]:
amazon_df

,title,price,rating,reviews,availability
0,Apple iPhone 15 Pro Max Silicone Case with Mag...,$49.00,4.3 out of 5 stars,"1,152 ratings",In Stock
1,Apple iPhone FineWoven Wallet with MagSafe - T...,$59.00,4.0 out of 5 stars,650 ratings,In Stock
2,"Apple iPhone 12, 64GB, Black - Fully Unlocked ...",$397.76,4.2 out of 5 stars,"17,083 ratings",Only 9 left in stock - order soon.
3,"Apple iPhone X, Fully Unlocked 5.8"", 256 GB - ...","$1,104.03",4.4 out of 5 stars,85 ratings,Temporarily out of stock.
4,Google Pixel 8 - Unlocked Android Smartphone w...,$549.00,4.3 out of 5 stars,579 ratings,In Stock
5,New Simple Mobile Prepaid - Apple iPhone SE (6...,$399.00,3.4 out of 5 stars,5 ratings,In Stock
6,"Apple iPhone 8 4.7"", 64 GB, Fully Unlocked, Si...",$705.02,4.2 out of 5 stars,89 ratings,Temporarily out of stock.
7,"Apple iPhone X, GSM Unlocked 5.8"", 256 GB - Sp...","$1,104.03",3.9 out of 5 stars,223 ratings,Temporarily out of stock.
8,"Apple iPhone 8 4.7"", 64 GB, GSM Unlocked, Spac...",$705.02,4.1 out of 5 stars,25 ratings,Temporarily out of stock.
9,"Apple iPhone 7 128 GB Unlocked, Silver Interna...",$649.00,4.3 out of 5 stars,82 ratings,Temporarily out of stock.
